# Use the best Auto-ML generated notebook to bootstrap our ML Project

<img src="https://raw.githubusercontent.com/QuentinAmbard/databricks-demo/main/retail/resources/images/lakehouse-retail/lakehouse-retail-churn-ml-experiment.png" style="float: right" width="600px">

Databricks Auto-ML tries many models and generate notebooks containing the code used to build the model.

Here, we selected the notebook from best run from the Auto ML experiment.

All the code below has been automatically generated. As Data Scientist, we can tune it based on our business knowledge, or use the model generated as it is.

This saves Datascientists hours of developement and allow team to quickly bootstrap and validate new project.

*Make sure you run the previous notebook to be able to access the data.*

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F04-Data-Science-ML%2F04.2-automl-generated-notebook-iot-turbine&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F04-Data-Science-ML%2F04.2-automl-generated-notebook-iot-turbine&version=1">

In [0]:
%run ../config

In [0]:
%run ../_resources/00-global-setup-v2

In [0]:

dbutils.widgets.dropdown("reset_all_data", "false", ["true", "false"], "Reset all data")
reset_all_data = dbutils.widgets.get("reset_all_data") == "true"

DBDemos.setup_schema(catalog, db, reset_all_data, volume_name)
volume_folder =  f"/Volumes/{catalog}/{db}/{volume_name}"

In [0]:
import mlflow
# import databricks.automl_runtime

from mlflow.tracking import MlflowClient

mlflow.set_registry_uri("databricks-uc")

#Added for the demo purpose
xp = DBDemos.get_last_experiment(experiment_name)
#xp = DBDemos.get_last_experiment('navy-turbine-maintenance')

# Use MLflow to track experiments
mlflow.set_experiment(xp["path"])

#Run containing the data analysis notebook to get the data from artifact
# data_run = mlflow.search_runs(filter_string="tags.mlflow.source.name='Notebook: DataExploration'", order_by=['attributes.start_time DESC']).iloc[0].to_dict()

#get best run id (this notebook)
df = mlflow.search_runs(filter_string="metrics.val_f1_score < 1")
run = df.sort_values(by="metrics.val_f1_score", ascending=False).iloc[0].to_dict()


client = MlflowClient()
#Get the best run
best_run = client.get_run(run["run_id"])

#register best run
latest_model = mlflow.register_model(f'runs:/{best_run.info.run_id}/model', f"{catalog}.{db}.{model_name}")
  # Flag it as Production ready using UC Aliases
client.set_registered_model_alias(name=f"{catalog}.{db}.{model_name}", alias="prod", version=latest_model.version)



In [0]:
# from databricks.sdk import WorkspaceClient

# w = WorkspaceClient()

# PIPE_ID = "0b28ba35-59b7-4c9c-9aca-c59a78e34432"
# TABLES = [
#   'current_status_predictions',
#   'ship_current_status_gold'
# ]

# w.pipelines.start_update(pipeline_id=PIPE_ID, refresh_selection=TABLES)

We just moved our model as production ready! [Open the dbdemos_turbine_maintenance model](#mlflow/models/dbdemos_turbine_maintenance) to explore its artifact and analyze the parameters used, including traceability to the notebook used for its creation.